In [342]:
import nltk
from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()

import numpy as np 
import tflearn
import tensorflow as tf
import random

In [343]:
import json
with open('data.json') as json_data:
    intents = json.load(json_data)
    

In [344]:
lines = []
with open('./train_sentences/train_emotion.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        str,emo = line.split(';')
        if 'sadness' in emo: 
                if str not in intents['intents'][3]['patterns'] and len(intents['intents'][3]['patterns']) <= 400:
                        intents['intents'][3]['patterns'].append(str)
        if 'joy' in emo: 
                if str not in intents['intents'][4]['patterns']and len(intents['intents'][4]['patterns']) <= 400:
                        intents['intents'][4]['patterns'].append(str)
        if 'anger' in emo: 
                if str not in intents['intents'][5]['patterns']and len(intents['intents'][5]['patterns']) <= 400:
                        intents['intents'][5]['patterns'].append(str)
        if 'fear' in emo: 
                if str not in intents['intents'][6]['patterns']and len(intents['intents'][6]['patterns']) <= 400:
                        intents['intents'][6]['patterns'].append(str)


In [345]:
lines = []
with open('./train_sentences/train_greet.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        if line not in intents['intents'][0]['patterns']:
            intents['intents'][0]['patterns'].append(line)

lines = []
with open('./train_sentences/train_bye.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        if line not in intents['intents'][1]['patterns']:
            intents['intents'][1]['patterns'].append(line)

lines = []
with open('./train_sentences/train_thanks.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        if line not in intents['intents'][2]['patterns']:
            intents['intents'][2]['patterns'].append(line)


with open('data2.json', 'w') as outfile:
        json.dump(intents, outfile)


In [346]:
with open('data2.json') as json_data:
    intents = json.load(json_data)

In [347]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bz88k\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [348]:
words = []
classes = []
documents = []

ignore_words = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize words
        w = nltk.word_tokenize(pattern) 
         #add to words list
        words.extend(w)
         #add to documents
        documents.append((w,intent['tag']))
         #add to classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#stem and lower each word and remove duplicates

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#remove duplicates
classes = sorted(list(set(classes)))

# print(len(documents), "documents")
# print(len(classes), "classes", classes)
# print(len(words), "unique stemmed words", words)


In [349]:
training = []
outputt = []

#create an empty array for output
output_empty = [0] * len(classes)

#training set, BOW for each sentence
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])


C:\Users\bz88k\AppData\Local\Temp/ipykernel_17376/1583013068.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [350]:
tf.compat.v1.reset_default_graph() 

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation = 'softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir = 'tflearn_logs')
model.fit(train_x, train_y, n_epoch = 30, batch_size = 8, show_metric = True)
model.save('./model.tflearn')

Training Step: 6689  | total loss: 1.44869 | time: 0.364s
| Adam | epoch: 030 | loss: 1.44869 - acc: 0.9134 -- iter: 1776/1781
Training Step: 6690  | total loss: 1.30496 | time: 0.366s
| Adam | epoch: 030 | loss: 1.30496 - acc: 0.9220 -- iter: 1781/1781
--
INFO:tensorflow:d:\Waterloo Assignments\SE101\Chairapy\Chat\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [351]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [352]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 ... 0 0 0]
['anger', 'fear', 'goodbye', 'greeting', 'joy', 'sad', 'thanks']


In [353]:
print(model.predict([p]))

[[6.7762768e-01 3.3444486e-10 1.0761922e-04 2.6702943e-01 6.8534985e-07
  4.1336715e-03 5.1100872e-02]]


In [354]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [355]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [357]:
s = input()
print(response(s))

[insert response for anger]
None
